In [ ]:
rm -rf obitools4
git clone https://github.com/metabarcoding/obitools4 

Cloning into 'obitools4'...
remote: Enumerating objects: 7574, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 7574 (delta 287), reused 384 (delta 148), pack-reused 6983 (from 2)
Receiving objects: 100% (7574/7574), 188.68 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (4721/4721), done.
Updating files: 100% (529/529), done.


In [ ]:
### Fixer la version 
cd obitools4
git reset --hard 996ec69
cd ..

HEAD is now at 996ec69 update the release notes for version 4.4.0


In [ ]:
conda install -c conda-forge compilers=1.10 make=4.4.1 go=1.24.5 zlib=1.3.1 jq=1.8.1 --yes

Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [ ]:
# CHANGER LE LIEN DU GITLAB EN GITHUB
find . -type f -exec sed -i \
  -e 's|git\.metabarcoding\.org/obitools/obitools4/obitools4|github.com/metabarcoding/obitools4|g' \
  -e 's|git\.metabarcoding\.org/org-asm/org-asm\.git|github.com/metabarcoding/org-asm.git|g' \
  -e 's|git\.metabarcoding\.org/lecasofts/go/obitools|github.com/metabarcoding/obitools4|g' \
  -e 's|https://git\.metabarcoding\.org|https://github.com/metabarcoding|g' \
  {} +


In [ ]:
export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-cc
export CGO_ENABLED=1
export CGO_CFLAGS="-I$CONDA_PREFIX/include"
export CGO_LDFLAGS="-L$CONDA_PREFIX/lib"

In [ ]:
# Installation d'obitools V4
cd obitools4
make
cd ..

In [ ]:
## FORMATAGE DES DONNES / ESPACES RETIRE
shopt -s nullglob
for f in galaxy_inputs/inputfile/*.{fastq*,fasta}; do
  [ -e "$f" ] || continue
  dir=$(dirname -- "$f")
  base=$(basename -- "$f")
  # on enlève les espaces
  newbase=${base// /_}
  newpath="$dir/$newbase"
  [ "$newpath" = "$f" ] && continue
  echo "Renomme: $f -> $newpath"
  mv -- "$f" "$newpath"
done

In [ ]:
# Recherche des chemins des inputs et formatage de l'ouput 
jq '.inputfile[0].path |= gsub(" "; "_")' galaxy_inputs/galaxy_inputs.json >  galaxy_inputs/galaxy_inputs_fixed.json
json_path="galaxy_inputs/galaxy_inputs_fixed.json"

# input_file="galaxy_inputs/wolfdiet_obigrep.fastq" test injection d'input
input_file=$(grep -o '"path": *"[^"]*"' "$json_path" | head -n 1 | sed 's/.*"path": *"\([^"]*\)".*/\1/')


echo "Input file path: $input_file"

filename=$(basename -- "$input_file")       # ex: myfile.fastq.gz
format="${filename##*.}"                # ex: gz
filename="${filename%.*}"                  # ex: myfile.fastq ou myfile


echo "Nom de fichier (sans chemin, sans extension) : $filename"
echo "Extension détectée : $format"

Input file path: galaxy_inputs/wolfdiet_obigrep.fastq
Nom de fichier (sans chemin, sans extension) : wolfdiet_obigrep
Extension détectée : fastq


In [ ]:
# Gestion des format
changefastq=$(jq -r '.changefastq' $json_path)

changefasta=$(jq -r '.changefasta' $json_path)

In [ ]:
if [[ "$changefasta" == "true" ]]; then
    format="fasta"
    output_file="outputs/outputfile.$format"
    changefasta="--fasta-output"
else 
    changefasta=""
fi

In [ ]:
## Argument hasattrib with a option for the multiples arguments

# multiple_output
multiple_output=$(jq -r '.multipleXoutput' galaxy_inputs/galaxy_inputs.json)
# Input key 
key_hasattrib=$(jq -r '.keyXhasattrib' galaxy_inputs/galaxy_inputs.json)
#Input value 
value_hasattrib=$(jq -r '.valueXhasattrib' galaxy_inputs/galaxy_inputs.json)


# Transformer la chaîne en tableau
hasattrib=()
if [[ "$multiple_output" == "true" ]]; then 
    IFS=',' read -r -a value_hasattrib <<< "$value_hasattrib"
    for i in "${value_hasattrib[@]}"; do
        hasattrib+=("-a ${key_hasattrib}=${i}")
    done
else
    if [[ -n "$key_hasattrib" && "$key_hasattrib" != "null" ]]; then
        hasattrib+=("-a ${key_hasattrib}=${value_hasattrib}") 
    else 
        hasattrib=""
    fi
fi

In [ ]:
#### Variables
listarg=(seq def ident attrib predicat)

for i in "${listarg[@]}"; do
    if [[ -d "galaxy_inputs/$i" ]]; then
        value=$(head -n 1 "galaxy_inputs/$i"/*.txt)  # lit la 1ère ligne du fichier .txt dans ce dossier
        declare "$i=$value"                          # crée une variable dynamique nommée comme $i
    fi
done


# Test d'affichage
echo "seq: $seq"
echo "def: $def"
echo "ident: $ident"
echo "attrib: $attrib"
echo "predicat: $predicat"


# lmax=""
lmax=$(jq -r '.lmax' galaxy_inputs/galaxy_inputs.json)

# lmin=""   
lmin=$(jq -r '.lmin' galaxy_inputs/galaxy_inputs.json)
####


echo "$input_file"
echo "$seq"

seq: 
def: 
ident: 
attrib: 
hasattrib: 
predicat: 
bash: jq: command not found
bash: jq: command not found




In [ ]:
### Ajout de la particule de l'argument 
if [[ -n "$seq"  && "$seq" != "null" ]]; then
    seq="-s $seq"
    echo "variable seq présente: $seq"
else 
    seq=""
fi

if [[ -n "$def"  && "$def" != "null"  ]]; then
    def="-D $def"
    echo "variable def présente : $def"
else 
    def=""
fi

if [[ -n "$ident"  && "$ident" != "null" ]]; then
    ident="-I $ident"
    echo "variable ident présente : $ident"
else 
    ident=""
fi

if [[ -n "$idlist" && "$idlist" != "null" ]]; then
    idlist="--id-list $idlist"
    echo "variable idlist présente : $idlist"
else 
    idlist=""
fi

if [[ -n "$attrib" && "$attrib" != "null" ]]; then
    attrib="-A $attrib"
    echo "variable attrib présente : $attrib"
else 
    attrib=""
fi

if [[ -n "$predicat"  && "$predicat" != "null" ]]; then
    predicat="-p $predicat"
    echo "variable predicat présente : $predicat"
else 
    predicat=""
fi

if [[ -n "$lmax"  && "$lmax" != "null" ]]; then
    lmax="-L $lmax"
    echo "variable lmax présente : $lmax"
else 
    lmax=""
fi

if [[ -n "$lmin"  && "$lmin" != "null" ]]; then
    lmin="-l $lmin"
    echo "variable lmin présente : $lmin"
else 
    lmin=""
fi

maxcount=$(jq -r '.maxcount' galaxy_inputs/galaxy_inputs.json)
if [[ -n "$maxcount" && "$maxcount" != "null" ]]; then
    maxcount="-C $maxcount"
    echo "variable maxcount présente : $maxcount"
else 
    maxcount=""
fi  

mincount=$(jq -r '.mincount' galaxy_inputs/galaxy_inputs.json)
if [[ -n "$mincount" && "$mincount" != "null" ]]; then
    mincount="-c $mincount"
    echo "variable mincount présente : $mincount"
else
    mincount=""
fi      


variable predicat présente : -p mode!="joined"


In [ ]:
if [[ $multiple_output == "true" ]]; then
    for idx in "${!hasattrib[@]}"; do
        output_file="outputs/output_${value_hasattrib[$idx]}.$extension"
        ./obitools4/build/obigrep $seq $def $ident $idlist $attrib ${hasattrib[$idx]} $predicat $lmax $lmin $maxcount $mincount $input_file > $output_file $changefasta
    done
else 
    ./obitools4/build/obigrep $seq $def $ident $idlist $attrib $hasattrib $predicat $lmax $lmin $maxcount $mincount $input_file > $output_file $changefasta --no-order
fi

INFO[0000] Number of workers set 125                    
INFO[0000] Found 1 files to process                     
INFO[0000] galaxy_inputs/wolfdiet_obigrep.fastq mime type: text/fastq 
INFO[0000] On output use JSON headers                   
| Reading sequences (41435/-, 9924 it/s) [4s]  
